In [ ]:
import pandas as pd
import category_encoders as ce
from geopy.distance import geodesic

hotel_address — адрес отеля;
review_date — дата, когда рецензент разместил соответствующий отзыв;
average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
hotel_name — название отеля;
reviewer_nationality — страна рецензента;
negative_review — отрицательный отзыв, который рецензент дал отелю;
review_total_negative_word_counts — общее количество слов в отрицательном отзыв;
positive_review — положительный отзыв, который рецензент дал отелю;
review_total_positive_word_counts — общее количество слов в положительном отзыве.
reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
total_number_of_reviews — общее количество действительных отзывов об отеле;
tags — теги, которые рецензент дал отелю;
days_since_review — количество дней между датой проверки и датой очистки;
additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
lat — географическая широта отеля;
lng — географическая долгота отеля.

In [ ]:
hotels = pd.read_csv('..\data\hotels.csv')
hotels.head(3)

In [ ]:
hotels.info()

In [ ]:
hotels['review_year'] = pd.to_datetime(hotels['review_date']).dt.year
hotels['review_month'] = pd.to_datetime(hotels['review_date']).dt.month

hotels['days_since_review'] = hotels['days_since_review'].str.replace('days', '').str.replace('day', '').astype('int32')
hotels.drop('review_date', axis=1, inplace=True);


In [ ]:
def clear_empty_review(x, list_empty_words):
    striped_x = x.strip()
    lower_x = striped_x.lower()
    for word in list_empty_words:
        if lower_x.startswith(word):
            return ''
    return striped_x

def clear_negative_review(x):
    list_empty_words = ['nothing', 'n a', 'no negative', 'none', 'all good', 'everything was perfect', 'everything was great',
                        'everything was good', 'na', 'i liked everything', 'no complaints', 'can t think of anything', 'nil',
                        'liked everything', 'no', 'absolutely nothing', 'everything was fine', 'all was good', 
                        'there was nothing i didn t like', 'there was nothing to dislike', 'we liked everything',
                        'it was all good', 'loved everything', 'loved everything', 'all ok', 'liked it all', 'not much',
                        'there was nothing we didn t like']
    return clear_empty_review(x, list_empty_words)
 
def clear_positive_review(x):
    list_empty_words = ['nothing', 'n a', 'no positive', 'none', 'na',  'not much', ]
    return clear_empty_review(x, list_empty_words)   
        
hotels['negative_review'] = hotels['negative_review'].apply(clear_negative_review)
hotels['review_total_negative_word_counts'] = hotels['negative_review'].apply(lambda x: 0 if x=='' else x)
hotels['positive_review'] = hotels['positive_review'].apply(clear_positive_review)
hotels['review_total_positive_word_counts'] = hotels['positive_review'].apply(lambda x: 0 if x=='' else x)

In [ ]:
hotels['breakfast'] = hotels.apply(lambda x: -1 if 'breakfast' in x.negative_review else 1 if 'breakfast' in x.positive_review else 0, axis=1)
hotels['staff'] = hotels.apply(lambda x: -1 if 'staff' in x.negative_review else 1 if 'staff' in x.positive_review else 0, axis=1)
hotels['room'] = hotels.apply(lambda x: -1 if 'room' in x.negative_review else 1 if 'room' in x.positive_review else 0, axis=1)
hotels['restaurant'] = hotels.apply(lambda x: -1 if 'restaurant' in x.negative_review else 1 if 'restaurant' in x.positive_review else 0, axis=1)
hotels['expensive'] = hotels.apply(lambda x: -1 if 'expensive' in x.negative_review or 'price' in x.negative_review or 'cost' in x.negative_review else 1 if 'cheap' in x.positive_review else 0, axis=1)
hotels['location'] = hotels.apply(lambda x: -1 if 'location' in x.negative_review else 1 if 'location' in x.positive_review else 0, axis=1)
hotels['parking'] = hotels.apply(lambda x: -1 if 'parking' in x.negative_review else 1 if 'parking' in x.positive_review else 0, axis=1)

hotels.drop(['negative_review', 'positive_review'], axis=1, inplace=True);

In [ ]:
hotels['business_trip'] = hotels['tags'].apply(lambda x: 1 if 'Business trip' in x else 0)
hotels['leisure_trip'] = hotels['tags'].apply(lambda x: 1 if 'Leisure trip' in x else 0)
hotels['with_pet'] = hotels['tags'].apply(lambda x: 1 if 'With a pet' in x else 0)
hotels['tags'] = hotels['tags'].apply(lambda x: x.replace("' Business trip ',", '').replace("' Leisure trip ',", '').replace("' With a pet ',", ''))
hotels['tags_list'] = hotels['tags'].apply(lambda x: x.replace('[', '').replace(']', '').split(" ', ' "))
hotels['company_type'] = hotels['tags_list'].apply(lambda x: x[0].replace("'", '').strip() if x else '')
hotels['room_type'] = hotels['tags_list'].apply(lambda x: x[1].strip() if len(x)>1 else '')
hotels['nights_count'] = hotels['tags_list'].apply(lambda x: int(x[2].strip()[7:9]) if len(x)>2 else 0)

hotels.drop(['tags', 'tags_list'], axis=1, inplace=True);


In [ ]:
encoder = ce.OneHotEncoder(cols=['company_type']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(hotels['company_type'])
# jj = encoder.get_feature_names_out()
type_bin = type_bin.rename(columns={'company_type_1':'Couple','company_type_2':'Solo_traveler','company_type_3':'family_with_young_children','company_type_4':'Group','company_type_5':'family_with_old_children','company_type_6':'travelers_with_friends'})
hotels = pd.concat([hotels, type_bin], axis=1)
hotels.drop(['company_type'], axis=1, inplace=True);

In [ ]:
hotels['country'] = hotels['hotel_address'].apply(
    lambda x: x.split()[-1] 
        if x.split()[-1] != 'Kingdom'
        else ' '.join(x.split()[-2:])
)
    
hotels['city'] = hotels.apply(
    lambda x: x['hotel_address'].split()[-5] 
        if x['country'] == 'United Kingdom'
        else x['hotel_address'].split()[-2], axis=1
)

In [ ]:
population_df = pd.DataFrame(
    [
        ['London', 8.982, 51.50735, -0.12776], 
        ['Barcelona', 1.62, 41.3888, 2.15899], 
        ['Paris', 2.161, 48.85661, 2.35222], 
        ['Amsterdam', 0.822, 52.37022, 4.89517], 
        ['Vienna', 1.897, 48.20817, 48.20817], 
        ['Milan', 1.352, 45.45863, 9.18187]
    ], 
    columns=['city', 'city_population', 'center_lat', 'center_lng']
)
hotels = hotels.merge(population_df, how='left', on='city')
hotels['center_distance'] = hotels.apply(lambda x: x if pd.isna(x.lat) else geodesic((x.lat, x.lng), (x.center_lat, x.center_lng), ellipsoid='WGS-84').km, axis=1)
median_distance_df = hotels.groupby('city', as_index=False)['center_distance'].median()
hotels = hotels.merge(median_distance_df, how='left', on='city', suffixes=('', 'median'))
hotels['center_distance'] = hotels['center_distance'].fillna(hotels['center_distance_median'])
hotels.drop(['lat', 'lng', 'center_lat', 'center_lng'], axis=1, inplace=True);

In [ ]:
encoder = ce.OneHotEncoder(cols=['city']) # указываем столбец для кодирования
type_bin = encoder.fit_transform(hotels['city'])
type_bin = type_bin.rename(columns={'city_1':'London','city_2':'Paris','city_3':'Amsterdam','city_4':'Milan','city_5':'Vienna','city_6':'Barcelona'})
hotels = pd.concat([hotels, type_bin], axis=1)
hotels.drop(['city', 'country'], axis=1, inplace=True);

In [ ]:
nationality_df = hotels.groupby('hotel_name', as_index=False)['reviewer_nationality'].nunique()
nationality_df = nationality_df.rename(columns={'reviewer_nationality': 'reviewer_nationality_count'})
hotels = hotels.merge(nationality_df, how='left', on='hotel_name')
hotels.drop(['reviewer_nationality'], axis=1, inplace=True);

In [ ]:
hotels.info()

In [ ]:
# убираем признаки которые еще не успели обработать, 
# модель на признаках с dtypes "object" обучаться не будет, просто выберем их и удалим
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
hotels.drop(object_columns, axis = 1, inplace=True)

# заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [ ]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

Небольшой бонус:


In [ ]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)